<a href="https://colab.research.google.com/github/MohammedNasserAhmed/AINARABIC/blob/main/lab_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install firecrawl anthropic beautifulsoup4 python-dotenv


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 891.5/891.5 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.3/157.3 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.9 MB/s eta 0:00:00


In [2]:
import os
import time
from firecrawl import FirecrawlApp
import json
import anthropic
from bs4 import BeautifulSoup
from datetime import datetime
from typing import Optional, List
import re
import textwrap
from IPython.display import Markdown
from dotenv import load_dotenv
load_dotenv()

anthropic_api_key = os.getenv("ANTHROPIC_API_KEY") or ""
firecrawl_api_key = os.getenv("FIRECRAWL_API_KEY") or ""

In [3]:
import sys
sys.stdout.encoding = 'utf-8'

In [4]:
url=u"https://www.aljazeera.net/opinions/2024/3/9/%D8%A7%D9%84%D8%B0%D9%83%D8%A7%D8%A1-%D8%A7%D9%84%D8%A7%D8%B5%D8%B7%D9%86%D8%A7%D8%B9%D9%8A-%D8%A3%D8%A8%D8%B9%D8%AF-%D9%85%D9%86-%D9%83%D9%88%D9%86%D9%87-%D9%85%D8%AC%D8%B1%D8%AF"

In [10]:
client=anthropic.Client(api_key=anthropic_api_key)
app = FirecrawlApp(api_key=firecrawl_api_key)

In [11]:
scrape_result = app.scrape_url(url, params={'formats': ['markdown', 'html']})

In [12]:
def extract_main_content_from_html(html: str) -> str:
        soup = BeautifulSoup(html, 'html.parser')

        # Remove script and style elements
        for script in soup(["script", "style"]):
            script.decompose()

        # Find the main content container
        main_content = soup.select_one('main#main-content-area')

        if main_content:
            # Remove unwanted elements (adjust as needed)
            for unwanted in main_content.select('.article-info-block,.disclaimer-text, .article-author, .article-dates'):
                unwanted.decompose()

            # Extract text from all elements that might contain content
            content = []
            for element in main_content.descendants:
                if element.name == 'p':
                    content.append(element.get_text().strip())

            # Join all content and clean up
            full_content = '\n'.join(content)
            full_content = re.sub(r'\s+', ' ', full_content)  # Replace multiple spaces with single space
            full_content = re.sub(r'\n+', '\n', full_content)  # Replace multiple newlines with single newline

            return full_content.strip()

        return ""

def extract_author_from_html(html: str) -> Optional[str]:
        soup = BeautifulSoup(html, 'html.parser')
        author_element = soup.select_one('.article-author__name .author-link')
        if author_element:
            return author_element.text.strip()
        return None

def extract_date_from_html(html: str) -> Optional[datetime.date]:
    soup = BeautifulSoup(html, 'html.parser')
    date_element = soup.select_one('.article-dates__published')
    if date_element:
        date_str = date_element.text.strip()
        try:
            # Assuming the date format is always DD/MM/YYYY
            return datetime.strptime(date_str, '%d/%m/%Y').date()
        except ValueError:
            print(f"Error parsing date: {date_str}")
            return None
    return None


def save(data):
    with open('output.md', 'w', encoding='utf-8') as file:
        file.write(data)

    # Save markdown as .json file
    with open('output.json', 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

def format_data_dict_to_markdown(data_dict):
  markdown_text = ""
  for key, value in data_dict.items():
    if value:
      markdown_text += f"**{key}:** {value}\n"
  return markdown_text

def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))


In [18]:
def extract_author(markdown: str):
    # Pattern to match Arabic names, potentially preceded by titles like "د."
    author_pattern = r'\*\s*(د\.\s*)?[\u0600-\u06FF\s]+\n'
    author_match = re.search(author_pattern, markdown)
    if author_match:
        author = author_match.group().strip('* \n')
        return author.strip()
    return None

def extract_date(text):
    # This is a simple date extraction. You might need to adjust it based on the actual date format in your content.
    date_match = re.search(r'\d{4}-\d{2}-\d{2}', text)
    if date_match:
        return datetime.strptime(date_match.group(), '%Y-%m-%d').date()
    return None

if scrape_result:
    print('Collecting data from crawl results:\n')
    html = scrape_result.get('html')
    markdown = scrape_result.get('markdown')
    metadata = scrape_result.get('metadata')
    srcurl = metadata.get('ogUrl')
    title = metadata.get('ogTitle')
    description = metadata.get('ogDescription')
    site=metadata.get('ogSiteName')
    clean_content = extract_main_content_from_html(html)
    date = extract_date_from_html(html)
    author = extract_author_from_html(html)
    data_dict = {
            'SourceURL': srcurl,
            'Title': title,
            'Description': description,
            'Date': str(date),
            'Author': author,
            'Site': site,
            'MainContent': clean_content
        }
    data = format_data_dict_to_markdown(data_dict)
    save(data)
    print(f'Source URL: {data_dict["SourceURL"]}')
    print(f'Title: {data_dict["Title"]}')
    print(f'Description: {data_dict["Description"]}')
    print(f'Date: {data_dict["Date"]}')
    print(f'Author: {data_dict["Author"]}')
    print(f'Site: {data_dict["Site"]}')
    print('Main Content: \n')
to_markdown(clean_content)




Source URL: https://www.aljazeera.net/opinions/2024/3/9/%d8%a7%d9%84%d8%b0%d9%83%d8%a7%d8%a1-%d8%a7%d9%84%d8%a7%d8%b5%d8%b7%d9%86%d8%a7%d8%b9%d9%8a-%d8%a3%d8%a8%d8%b9%d8%af-%d9%85%d9%86-%d9%83%d9%88%d9%86%d9%87-%d9%85%d8%ac%d8%b1%d8%af
Title: الذكاء الاصطناعي أبعد من كونه مجرد أعجوبة تكنولوجية
Description: العالم دخل فعليًا مرحلة جديدة عنوانها السباق في تكنولوجيا الذكاء الاصطناعي، لكن إذا لم يتم وضع قواعد ناظمة لهذا السباق؛ فإنّه قد يخرج عن السيطرة، ويهدد المعمورة كلها.
Date: 2024-03-09
Author: د. خالد وليد محمود
Site: الجزيرة نت
Main Content: 



> مما لا شكّ فيه فقد أصبح الذكاء الاصطناعي الشغل الشاغل لغالبية حكومات الدول المتقدّمة؛ لإدراكها أن العالم يقف عند فجر حقبة جديدة، ستغيّر حياة البشرية والطريقة التي تعيش وتعمل بها في عدد كبير من المجالات والقطاعات المختلفة، مع الإقرار في نفس الوقت بأن مخاطر هذه الحقبة لا تزال مجهولة أيضًا. فلا مبالغة بالقول إن العالم دخل فعليًا مرحلة جديدة عنوانها السباق في تكنولوجيا الذكاء الاصطناعي، لكن إذا لم يتم وضع قواعد ناظمة لهذا السباق؛ فإنّه قد يخرج عن السيطرة، ويهدد المعمورة كلها؛ لأنه يثير العديد من المخاطر والتهديدات الأمنية والأخلاقية؛ نتيجة الاعتماد المتزايد عليه. ليس صدفة أن أكبر الاستثمارات فيه التي يقوم بها العملاقان المتنافسان، الولايات المتحدة والصين، هي في هذا المجال كما أن التطبيقات الضخمة لهذا الذكاء، تعد تمهيدًا لحرب ناعمة بين الآلة والإنسان، ولا يمكن قراءة استقالة "جيفري هينتون" – الذي يطلق عليه "الأب الروحي للذكاء الاصطناعي" من شركة غوغل أوائل مايو 2023 – إلا في سياق "جرس الإنذار" الذي ينبّه البشرية إلى تلك المخاطر العديدة المتعلّقة بالذكاء الاصطناعي. التقديرات تشير إلى أن حوالي 800 مليون شخص سيفقدون وظائفهم خلال الأعوام المقبلة؛ بسبب تطوير الشركات التكنولوجية العملاقة جيلًا جديدًا من البشر الرقْميّين الذين سيتفوقون على العُمّال والمُحترفين من البشر، بل يتوقّع بعض العلماء أن يحلّ الذكاء الاصطناعي، مكان البشر في 99% من الوظائف في غضون العقدين المقبِلين. المعروف أن الذكاء الاصطناعي ليس قضية اقتصادية فقط، ولا هو مجرد الروبوت – وأليكسا وأمازون – الذي يخدمك في منزلك أو يراجع جدول الضرب مع أطفالك، أو تلك التكنولوجيا التي تؤسّس لثورة طبية جديدة، أو التعليم والصناعة، أو المركبات التي تعمل من دون سائق؛ بل إنه أيضًا أداة سيطرة وهمينة وفرض نفوذ من خلال التكنولوجيا العسكرية والدفاعية الصاعدة. إذ ليس صدفة أن أكبر الاستثمارات فيه التي يقوم بها العملاقان المتنافسان، الولايات المتحدة والصين، هي في هذا المجال. فالعالم بأجمعه يعرف غوغل، ومايكروسوفت، وأمازون، والبعض فقط يعرف علي بابا، وبايدو، وتنسنت الصينية المهمة بالقدر نفسه، هذا دون احتساب مراكز البحث المتخصصة المرتبطة بها. قبل عقدين من الزمان كان الحديث عن طموحات الذكاء الاصطناعي والروبوتات بديلًا للعقل البشري – أو على الأقل تدريب الآلة لتنفّذ ما يقوم بعمله الإنسان أو العامل أو الموظف في مناحي الحياة – يقابل إما بالاستخفاف والتقليل من الأهمية من غير المتخصصين والمهتمين، بينما كان يرى الجانب الآخر من المتخصصين، أن الأمر لا يزيد على مجرد ضرب من ضروب الخيال العلمي أو الفانتازيا الهوليودية. اليوم يترقب العالم بحذر تطورات التنافس في هذا المجال "الذكاء الاصطناعي"؛ لأنّ من سيقوده، سيمسك بزمام الثورة الصناعية الخامسة. في ظل اقترابنا من بداية حقبة جديدة، يظهر مستقبل الذكاء الاصطناعي في الأفق بإمكانات واسعة وعمق متنوع. في هذا السياق المتسارع التطور، أصبح الذكاء الاصطناعي ليس مجرد تكنولوجيا مدهشة، بل أصبح ركيزة حاسمة للمستقبل الذي كان يومًا ما خيالًا علميًا. فمن الخوارزميات المتطورة التي تدعم تفاعلاتنا الرقمية اليومية، إلى العوالم الناشئة للمركبات ذاتية القيادة والروبوتات المتقدمة، يعيد الذكاء الاصطناعي تشكيل عالمنا بطرق فريدة. وبدلًا من كونه مجرد سرد للقدرات التكنولوجية، يمثل مستقبل الذكاء الاصطناعي تفاعلًا معقدًا يشمل الابتكار، وتأثيره الواسع في المجتمع، والمسؤولية الأخلاقية. ووفق هذا المشهد سريع التطور، لا يعد الذكاء الاصطناعي اليوم مجرد أعجوبة تكنولوجية – في ظل معطيات تتحدث عن توقعات بشأن وصول الإنفاق العالمي على أنظمة الذكاء الاصطناعي إلى حوالي 300 مليار دولار بحلول 2026 – بل أصبح مهندسًا محوريًا للمستقبل الذي كان ذات يوم عالمًا من الخيال العلمي. مع الأخذ بعين الاعتبار أن هناك طفرة واسعة في تقنيات وتطبيقات الذكاء الاصطناعي، ستؤدي إلى تحولات متسارعة في أسواق العمل، وتخلق مزيدًا من المخاوف بشأن المنافسة بين البشر وتلك التطبيقات، وسط تنافس وسباق محموم بين شركات عمالقة التكنولوجيا على تطوير تقنيات متقدمة، يُمكنها مواصلة الثورة التي يصنعها الذكاء الاصطناعي في الأسواق العالمية. ويبقى السؤال المعلق أين العرب من هذا المشهد؟ صحيح أن المؤشر العالمي للذكاء الاصطناعي من Tortoise Intelligence وضع كلًا من السعودية، والإمارات، وقطر، والبحرين في المراتب الأولى عربيًا بوصفها دولًا لها بصمة في هذا المجال، لكنّ كثيرًا من الدول العربية ما زالت خجولة في تبنّي الذكاء الاصطناعي، وكأنها منفصلة عن الواقع بمستويات مختلفة. إنّ الاقتصاد المستقبلي هو اقتصاد معرفي (تقني – رقمي)، وفي حال بقيت دول بعيدة عما يجري في عالم الذكاء الاصطناعي وتطبيقاته، فستكون منفصلة عن هذا الواقع، وبعيدة كل البعد عمّا يحصل من تطوّر على مستوى الثورة الصناعية الرابعة.. فهل نحن فاعلون؟

In [22]:
to_markdown(scrape_result.get('markdown'))

> *   [![د. خالد وليد محمود](https://www.aljazeera.net/wp-content/uploads/2000/01/fc0b5066-dc60-4378-bc48-43a5ef320b6a.jpeg?resize=96%2C96&quality=80)](/author/%d8%ae%d8%a7%d9%84%d8%af_%d9%88%d9%84%d9%8a%d8%af_%d9%85%d8%ad%d9%85%d9%88%d8%af)
>     
>     [د. خالد وليد محمود](/author/%d8%ae%d8%a7%d9%84%d8%af_%d9%88%d9%84%d9%8a%d8%af_%d9%85%d8%ad%d9%85%d9%88%d8%af)
>     
>     متخصص في السياسة السيبرانية
>     
> 
> 9/3/2024\-|آخر تحديث: 9/3/202412:07 ص (بتوقيت مكة المكرمة)
> 
> احفظ المقالات لقراءتها لاحقا وأنشئ قائمة قراءتك
> 
> إغلاق تلميح الأدوات
> 
> [facebook](https://www.facebook.com/sharer/sharer.php?u=https%3A%2F%2Faja.ws%2F5krdco)
> [twitter](https://twitter.com/intent/tweet?text=%D8%A7%D9%84%D8%B0%D9%83%D8%A7%D8%A1%20%D8%A7%D9%84%D8%A7%D8%B5%D8%B7%D9%86%D8%A7%D8%B9%D9%8A%20%D8%A3%D8%A8%D8%B9%D8%AF%20%D9%85%D9%86%20%D9%83%D9%88%D9%86%D9%87%20%D9%85%D8%AC%D8%B1%D8%AF%20%D8%A3%D8%B9%D8%AC%D9%88%D8%A8%D8%A9%20%D8%AA%D9%83%D9%86%D9%88%D9%84%D9%88%D8%AC%D9%8A%D8%A9&source=sharethiscom&related=sharethis&via=AJArabicnet&url=https%3A%2F%2Faja.ws%2F5krdco)
> [whatsapp](whatsapp://send?text=https%3A%2F%2Faja.ws%2F5krdco)
> [copylink](https://aja.ws/5krdco)
> 
> ![Artificial intelligence, futuristic digital technology human and robot face close up, digital smart world metaverse concept](https://www.aljazeera.net/wp-content/uploads/2023/10/fafsfasf-2-1698767641.jpg?resize=770%2C513&quality=80)
> 
> الذكاء الاصطناعي والتكنولوجيا الرقمية المستقبلية وجه الإنسان والروبوت عن قرب (شترستوك)
> 
> مما لا شكّ فيه فقد أصبح الذكاء الاصطناعي الشغل الشاغل لغالبية حكومات الدول المتقدّمة؛ لإدراكها أن العالم يقف عند فجر حقبة جديدة، ستغيّر حياة البشرية والطريقة التي تعيش وتعمل بها في عدد كبير من المجالات والقطاعات المختلفة، مع الإقرار في نفس الوقت بأن مخاطر هذه الحقبة لا تزال مجهولة أيضًا.
> 
> فلا مبالغة بالقول إن العالم دخل فعليًا مرحلة جديدة عنوانها السباق في تكنولوجيا الذكاء الاصطناعي، لكن إذا لم يتم وضع قواعد ناظمة لهذا السباق؛ فإنّه قد يخرج عن السيطرة، ويهدد المعمورة كلها؛ لأنه يثير العديد من المخاطر والتهديدات الأمنية والأخلاقية؛ نتيجة الاعتماد المتزايد عليه.
> 
> > ليس صدفة أن أكبر الاستثمارات فيه التي يقوم بها العملاقان المتنافسان، الولايات المتحدة والصين، هي في هذا المجال
> 
> كما أن التطبيقات الضخمة لهذا الذكاء، تعد تمهيدًا لحرب ناعمة بين الآلة والإنسان، ولا يمكن قراءة استقالة "جيفري هينتون" – الذي يطلق عليه "الأب الروحي للذكاء الاصطناعي" من شركة غوغل أوائل مايو 2023 – إلا في سياق "جرس الإنذار" الذي ينبّه البشرية إلى تلك المخاطر العديدة المتعلّقة بالذكاء الاصطناعي.
> 
> التقديرات تشير إلى أن حوالي 800 مليون شخص سيفقدون وظائفهم خلال الأعوام المقبلة؛ بسبب تطوير الشركات التكنولوجية العملاقة جيلًا جديدًا من البشر الرقْميّين الذين سيتفوقون على العُمّال والمُحترفين من البشر، بل يتوقّع بعض العلماء أن يحلّ الذكاء الاصطناعي، مكان البشر في 99% من الوظائف في غضون العقدين المقبِلين.
> 
> المعروف أن الذكاء الاصطناعي ليس قضية اقتصادية فقط، ولا هو مجرد الروبوت – وأليكسا وأمازون – الذي يخدمك في منزلك أو يراجع جدول الضرب مع أطفالك، أو تلك التكنولوجيا التي تؤسّس لثورة طبية جديدة، أو التعليم والصناعة، أو المركبات التي تعمل من دون سائق؛ بل إنه أيضًا أداة سيطرة وهمينة وفرض نفوذ من خلال التكنولوجيا العسكرية والدفاعية الصاعدة.
> 
> إذ ليس صدفة أن أكبر الاستثمارات فيه التي يقوم بها العملاقان المتنافسان، الولايات المتحدة والصين، هي في هذا المجال. فالعالم بأجمعه يعرف غوغل، ومايكروسوفت، وأمازون، والبعض فقط يعرف علي بابا، وبايدو، وتنسنت الصينية المهمة بالقدر نفسه، هذا دون احتساب مراكز البحث المتخصصة المرتبطة بها.
> 
> قبل عقدين من الزمان كان الحديث عن طموحات الذكاء الاصطناعي والروبوتات بديلًا للعقل البشري – أو على الأقل تدريب الآلة لتنفّذ ما يقوم بعمله الإنسان أو العامل أو الموظف في مناحي الحياة – يقابل إما بالاستخفاف والتقليل من الأهمية من غير المتخصصين والمهتمين، بينما كان يرى الجانب الآخر من المتخصصين، أن الأمر لا يزيد على مجرد ضرب من ضروب الخيال العلمي أو الفانتازيا الهوليودية.
> 
> اليوم يترقب العالم بحذر تطورات التنافس في هذا المجال "الذكاء الاصطناعي"؛ لأنّ من سيقوده، سيمسك بزمام الثورة الصناعية الخامسة.
> 
> في ظل اقترابنا من بداية حقبة جديدة، يظهر مستقبل الذكاء الاصطناعي في الأفق بإمكانات واسعة وعمق متنوع. في هذا السياق المتسارع التطور، أصبح الذكاء الاصطناعي ليس مجرد تكنولوجيا مدهشة، بل أصبح ركيزة حاسمة للمستقبل الذي كان يومًا ما خيالًا علميًا.
> 
> ### اشترك في
> 
> #### النشرة البريدية الأسبوعية: سياسة
> 
> حصاد سياسي من الجزيرة نت لأهم ملفات المنطقة والعالم.
> 
> اشترك الآن
> 
> ![](https://www.aljazeera.net/static/media/error-icon.c8fb9e1b.svg)حدث خطأ في الاشتراك، حاول مرة أخرى
> 
> ![](https://www.aljazeera.net/static/media/right-mark-icon.3a446adc.svg)الرجاء التحقق من بريدك الإلكتروني لتأكيد الاشتراك
> 
> عند قيامكم بالتسجيل، فهذا يعني موافقتكم على [سياسة الخصوصية للشبكة](https://privacy.aljazeera.net/ar/)
> 
> محمي بخدمة **reCAPTCHA**
> 
> فمن الخوارزميات المتطورة التي تدعم تفاعلاتنا الرقمية اليومية، إلى العوالم الناشئة للمركبات ذاتية القيادة والروبوتات المتقدمة، يعيد الذكاء الاصطناعي تشكيل عالمنا بطرق فريدة. وبدلًا من كونه مجرد سرد للقدرات التكنولوجية، يمثل مستقبل الذكاء الاصطناعي تفاعلًا معقدًا يشمل الابتكار، وتأثيره الواسع في المجتمع، والمسؤولية الأخلاقية.
> 
> ووفق هذا المشهد سريع التطور، لا يعد الذكاء الاصطناعي اليوم مجرد أعجوبة تكنولوجية – في ظل معطيات تتحدث عن توقعات بشأن وصول الإنفاق العالمي على أنظمة الذكاء الاصطناعي إلى حوالي 300 مليار دولار بحلول 2026 – بل أصبح مهندسًا محوريًا للمستقبل الذي كان ذات يوم عالمًا من الخيال العلمي.
> 
> مع الأخذ بعين الاعتبار أن هناك طفرة واسعة في تقنيات وتطبيقات الذكاء الاصطناعي، ستؤدي إلى تحولات متسارعة في أسواق العمل، وتخلق مزيدًا من المخاوف بشأن المنافسة بين البشر وتلك التطبيقات، وسط تنافس وسباق محموم بين شركات عمالقة التكنولوجيا على تطوير تقنيات متقدمة، يُمكنها مواصلة الثورة التي يصنعها الذكاء الاصطناعي في الأسواق العالمية.
> 
> ويبقى السؤال المعلق أين العرب من هذا المشهد؟ صحيح أن المؤشر العالمي للذكاء الاصطناعي من Tortoise Intelligence وضع كلًا من السعودية، والإمارات، وقطر، والبحرين في المراتب الأولى عربيًا بوصفها دولًا لها بصمة في هذا المجال، لكنّ كثيرًا من الدول العربية ما زالت خجولة في تبنّي الذكاء الاصطناعي، وكأنها منفصلة عن الواقع بمستويات مختلفة.
> 
> إنّ الاقتصاد المستقبلي هو اقتصاد معرفي (تقني – رقمي)، وفي حال بقيت دول بعيدة عما يجري في عالم الذكاء الاصطناعي وتطبيقاته، فستكون منفصلة عن هذا الواقع، وبعيدة كل البعد عمّا يحصل من تطوّر على مستوى الثورة الصناعية الرابعة.. فهل نحن فاعلون؟
> 
> الآراء الواردة في المقال لا تعكس بالضرورة الموقف التحريري لشبكة الجزيرة.
> 
> * * *